In [1]:
!pip install numpy==1.15

    100% |████████████████████████████████| 13.9MB 3.5MB/s 
datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.8 which is incompatible.
  Found existing installation: numpy 1.16.2
    Uninstalling numpy-1.16.2:
      Successfully uninstalled numpy-1.16.2


In [1]:
!pip install --upgrade pandas folium imgaug

    100% |████████████████████████████████| 10.1MB 5.3MB/s 
Requirement already up-to-date: folium in /usr/local/lib/python3.6/dist-packages (0.8.3)
Requirement already up-to-date: imgaug in /usr/local/lib/python3.6/dist-packages (0.2.8)
  Found existing installation: pandas 0.23.4
    Uninstalling pandas-0.23.4:
      Successfully uninstalled pandas-0.23.4


In [1]:
!pip3 install https://download.pytorch.org/whl/cu90/torch-1.0.0-cp36-cp36m-linux_x86_64.whl

    100% |████████████████████████████████| 591.8MB 23kB/s 
  Found existing installation: torch 1.0.1.post2
    Uninstalling torch-1.0.1.post2:
      Successfully uninstalled torch-1.0.1.post2


In [0]:
!pip3 install torchvision

In [0]:
!pip install pytorch-pretrained-bert==v0.6.1

    100% |████████████████████████████████| 122kB 4.6MB/s 


In [3]:
!git clone -b master https://github.com/charles9n/bert-sklearn

Cloning into 'bert-sklearn'...
remote: Enumerating objects: 107, done.
remote: Total 107 (delta 0), reused 0 (delta 0), pack-reused 107
Receiving objects: 100% (107/107), 378.81 KiB | 4.68 MiB/s, done.
Resolving deltas: 100% (44/44), done.


In [4]:
cd bert-sklearn

/content/bert-sklearn


In [5]:
!pip install .

Processing /content/bert-sklearn
    100% |████████████████████████████████| 122kB 4.5MB/s 
  Stored in directory: /root/.cache/pip/wheels/61/95/c6/5790aae8fb377f5ff356dbe58205aab28858595d6bff8197d0
Successfully built bert-sklearn


In [6]:
cd ..

/content


In [0]:
!dir

bert-sklearn  sample_data


In [7]:
import numpy as np
import pandas as pd
import os
import sys
import csv
from sklearn import metrics
from sklearn.metrics import classification_report

sys.path.append("../") 
from bert_sklearn import BertClassifier
from bert_sklearn import load_model

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [9]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 131304 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.3-0ubuntu3~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.3-0ubuntu3~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.3-0ubuntu3~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=ht

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
!mkdir -p glue_data

In [12]:
cd glue_data

/content/glue_data


In [0]:
!mkdir -p CoLA

In [14]:
cd CoLA

/content/glue_data/CoLA


In [15]:
from google.colab import files

uploaded = files.upload()

Saving dev.tsv to dev.tsv
Saving test.tsv to test.tsv
Saving train.tsv to train.tsv


In [0]:
!mkdir -p original

In [17]:
cd original

/content/glue_data/CoLA/original


In [0]:
!mkdir -p raw

In [19]:
cd raw

/content/glue_data/CoLA/original/raw


In [20]:
from google.colab import files

uploaded = files.upload()

Saving in_domain_dev.tsv to in_domain_dev.tsv
Saving in_domain_train.tsv to in_domain_train.tsv
Saving out_of_domain_dev.tsv to out_of_domain_dev.tsv


In [21]:
cd ..

/content/glue_data/CoLA/original


In [0]:
!mkdir -p tokenized

In [23]:
cd tokenized

/content/glue_data/CoLA/original/tokenized


In [24]:
from google.colab import files

uploaded = files.upload()

Saving in_domain_dev.tsv to in_domain_dev.tsv
Saving in_domain_train.tsv to in_domain_train.tsv
Saving out_of_domain_dev.tsv to out_of_domain_dev.tsv


In [28]:
cd ..

/content


In [29]:
from google.colab import files

uploaded = files.upload()

Saving download_glue_data.py to download_glue_data.py


In [0]:
DATADIR = os.getcwd() + '/glue_data'

In [31]:
!python3 download_glue_data.py --data_dir glue_data --tasks CoLA 

	Completed!


In [32]:
"""
CoLA train data size: 8551 
CoLA dev data size: 1043 
"""

def get_cola_df(filename,cols = [3,1]):
    df = pd.read_csv(filename, sep='\t',  encoding = 'utf8',keep_default_na=False,header=None)
    df = df[cols]
    df.columns=['text','label']
    return df

def get_cola_data(train_file = DATADIR+'/CoLA/train.tsv', 
                   dev_file =  DATADIR+'/CoLA/dev.tsv'):
    
    train = get_cola_df(train_file)
    print("CoLA train data size: %d "%(len(train)))
    dev = get_cola_df(dev_file)
    print("CoLA dev data size: %d "%(len(dev)))

    label_list = np.unique(train['label'].values)
    return train,dev,label_list  
                  
train,dev,label_list =  get_cola_data()

CoLA train data size: 8551 
CoLA dev data size: 1043 


In [33]:
print(label_list)

[0 1]


In [0]:
train = train.sample(len(train))

In [35]:
train.head()

,text,label
6772,Because Dr Jones ate too much rich food didn't...,0
5427,John is the kind of a fool that I told you about.,1
4163,This Government has been more transparent in t...,1
3078,Ellen and Helen conferred.,1
6074,Calvin will eat.,1


In [36]:
from sklearn.model_selection import GridSearchCV

X_train = train['text']
y_train = train['label']



#params = {'learning_rate':[2e-5, 3e-5]} #'train_batch_size':[32, 64]
params = {'train_batch_size':[32, 64]}

# wrap classifier/regressor in GridSearchCV
clf = GridSearchCV(BertClassifier(epochs = 3), 
                    params,
                    scoring='accuracy',
                    verbose=True)

# fit gridsearch 
clf.fit(X_train ,y_train)


Building sklearn classifier...
Building sklearn classifier...
Fitting 3 folds for each of 2 candidates, totalling 6 fits
Building sklearn classifier...


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
100%|██████████| 231508/231508 [00:00<00:00, 2670550.77B/s]


Loading bert-base-uncased model...


100%|██████████| 407873900/407873900 [00:08<00:00, 47392307.30B/s]


Defaulting to linear classifier/regressor
train data size: 5130, validation data size: 570


...

Training: 100%|██████████| 161/161 [04:05<00:00,  1.28s/it, loss=0.113]
                                                           

Epoch 3, Train loss : 0.1127, Val loss: 0.6171, Val accy = 80.70%


Building sklearn classifier...
Loading bert-base-uncased model...
Defaulting to linear classifier/regressor
train data size: 5130, validation data size: 570


Training:   0%|          | 0/81 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:542: FutureWarning: From version 0.22, errors during fit will result in a cross validation score of NaN by default. Use error_score='raise' if you want an exception raised or error_score=np.nan to adopt the behavior from version 0.22.
  FutureWarning)


RuntimeError: ignored

In [0]:
# best scores
print(clf.best_score_, clf.best_params_)

means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']

for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))